In [10]:
import requests
import pprint
API_TOKEN = "FOO"

# Station metadata

The following cells show how station data may be requested from the API and loaded into a pandas dataframe.

In [12]:
# Test out whether it's working
# You will need to substitute your own API token
params = {
    "token": API_TOKEN,
    "country": "us",
    "state": "ca",
    "county": "San Diego",
    "limit": "3",
}
resp = requests.get("https://api.synopticdata.com/v2/stations/metadata", params=params)
pprint.pprint(resp.json())

{'STATION': [{'ELEVATION': '328',
              'ELEV_DEM': '315',
              'ID': '144',
              'LATITUDE': '33.13000',
              'LONGITUDE': '-117.27583',
              'MNET_ID': '1',
              'NAME': 'Carlsbad, McClellan-Palomar Airport',
              'PERIOD_OF_RECORD': {'end': '2020-01-24T19:30:00Z',
                                   'start': '1997-04-12T00:00:00Z'},
              'RESTRICTED': False,
              'STATE': 'CA',
              'STATUS': 'ACTIVE',
              'STID': 'KCRQ',
              'TIMEZONE': 'America/Los_Angeles'},
             {'ELEVATION': '2648',
              'ELEV_DEM': '2614.8',
              'ID': '151',
              'LATITUDE': '32.62611',
              'LONGITUDE': '-116.46833',
              'MNET_ID': '1',
              'NAME': 'Campo',
              'PERIOD_OF_RECORD': {'end': '2020-01-24T18:52:00Z',
                                   'start': '1998-07-14T00:00:00Z'},
              'RESTRICTED': False,
              '

In [14]:
import pandas as pd
stations = resp.json()['STATION']
df = pd.DataFrame(data=stations)

In [15]:
df.head()

,STATUS,MNET_ID,PERIOD_OF_RECORD,ELEVATION,NAME,STID,ELEV_DEM,LONGITUDE,STATE,RESTRICTED,LATITUDE,TIMEZONE,ID
0,ACTIVE,1,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",328,"Carlsbad, McClellan-Palomar Airport",KCRQ,315,-117.27583,CA,False,33.13000,America/Los_Angeles,144
1,ACTIVE,1,"{'start': '1998-07-14T00:00:00Z', 'end': '2020...",2648,Campo,KCZZ,2614.8,-116.46833,CA,False,32.62611,America/Los_Angeles,151
2,ACTIVE,1,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",420,"San Diego, Montgomery Field",KMYF,419.9,-117.13639,CA,False,32.81444,America/Los_Angeles,282


In [19]:
df['RECORD_START'] = df['PERIOD_OF_RECORD'].apply(lambda x: x['start']).astype('datetime64')
df['RECORD_END'] = df['PERIOD_OF_RECORD'].apply(lambda x: x['end']).astype('datetime64')
df.head()

,STATUS,MNET_ID,PERIOD_OF_RECORD,ELEVATION,NAME,STID,ELEV_DEM,LONGITUDE,STATE,RESTRICTED,LATITUDE,TIMEZONE,ID,RECORD_START,RECORD_END
0,ACTIVE,1,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",328,"Carlsbad, McClellan-Palomar Airport",KCRQ,315,-117.27583,CA,False,33.13000,America/Los_Angeles,144,1997-04-12,2020-01-24 19:30:00
1,ACTIVE,1,"{'start': '1998-07-14T00:00:00Z', 'end': '2020...",2648,Campo,KCZZ,2614.8,-116.46833,CA,False,32.62611,America/Los_Angeles,151,1998-07-14,2020-01-24 18:52:00
2,ACTIVE,1,"{'start': '1997-04-12T00:00:00Z', 'end': '2020...",420,"San Diego, Montgomery Field",KMYF,419.9,-117.13639,CA,False,32.81444,America/Los_Angeles,282,1997-04-12,2020-01-24 19:30:00


# Weather data

The following cells demonstrate getting weather data from a particular station.

In [53]:
import datetime
min_end = df.RECORD_END.min()
start = min_end - datetime.timedelta(hours=1)
stids = df.STID.value_counts().index
params = {
    "token": API_TOKEN,
    "start": start.strftime("%Y%m%d%H%M"),
    "end": min_end.strftime("%Y%m%d%H%M"),
    "stid": ",".join(stids),
}
resp = requests.get("https://api.synopticdata.com/v2/stations/timeseries", params=params)

In [54]:
pprint.pprint(resp.json())

{'SUMMARY': {'RESPONSE_CODE': 20,
             'RESPONSE_MESSAGE': 'Account usage has exceeded the free tier. '
                                 'Please see https://synopticdata.com/pricing/ '
                                 'for more options.',
             'RESPONSE_TIME': '0 ms'}}
